In [1]:
import pymongo
client = pymongo.MongoClient("localhost", 27017)


In [2]:
db = client.pyprotolinc
db.list_collection_names()

['jobs', 'results']

In [3]:
results_col = db.results
jobs_col = db.jobs

In [4]:
# delete all: 
#jobs_col.delete_many(filter={})
#results_col.delete_many(filter={})
for r in jobs_col.find({}, {""}):
    print(r)

{'_id': '0dc2a52c-ec0b-11ed-a46b-4c034f70a3f9', 'params': {'config_file': '12'}, 'events': [{'type': 'API_JOB_RECEIVED', 'state': 'TO_BE_SUBMITTED', 'timestamp': 1683376773.800887}], 'subtasks': {'d0893d9d-6f68-4f51-9384-09b88ccd88ce': {'task_id': 'd0893d9d-6f68-4f51-9384-09b88ccd88ce', 'root_task_id': 'd0893d9d-6f68-4f51-9384-09b88ccd88ce', 'name': 'pyprotolinc.server.tasks.valuation_run_modular', 'events': [{'hostname': 'worker1@DESKTOP-F633F5A', 'utcoffset': -2, 'pid': 663, 'clock': 2368, 'uuid': 'd0893d9d-6f68-4f51-9384-09b88ccd88ce', 'timestamp': 1683376773.937037, 'type': 'task-started', 'local_received': 1683376773.9412887, 'state': 'STARTED'}, {'hostname': 'worker1@DESKTOP-F633F5A', 'utcoffset': -2, 'pid': 663, 'clock': 2376, 'uuid': 'd0893d9d-6f68-4f51-9384-09b88ccd88ce', 'result': "'d0893d9d-6f68-4f51-9384-09b88ccd88ce'", 'runtime': 0.013890489000914386, 'timestamp': 1683376773.956936, 'type': 'task-succeeded', 'local_received': 1683376773.96328, 'state': 'SUCCESS'}]}, 'a7919

In [5]:
r["events"]

[{'type': 'API_JOB_RECEIVED',
  'state': 'TO_BE_SUBMITTED',
  'timestamp': 1683376773.800887}]

In [77]:

r["events"][-1]['timestamp']

1682935865.4772186

### Job List Query

In [200]:
q = jobs_col.aggregate([{"$project": {"latestEvent": {"$last": "$events"}, "firstEvent": {"$first": "$events"}}},
                            {"$project": {"_id": 0,
                                          "job_id": "$_id",
                                          "submitted_at": "$firstEvent.timestamp",
                                          "state": "$latestEvent.state",
                                          "timestamp": "$latestEvent.timestamp"}},
                            {"$sort": {"submitted_at": 1}},
                            ])

q = jobs_col.aggregate([{"$match": {"events.0.timestamp": {"$gte": 1683376773.800887}}},  # filter for started date
                        {"$project": {"_id": True, "job_started": {"$first": "$events"}, "subtasks": True}},
                        {'$addFields': {'subtasksarr': {"$objectToArray": '$subtasks'}}},
                        {'$project': {'subtasks_new': '$subtasksarr.v', "job_started": "$job_started.timestamp"}},
                        {'$project': {'subtasks': False, 'subtaskarr': False}},
                        {'$unwind': '$subtasks_new'},
                        {"$project": {"_id": True,
                                      'name': "$subtasks_new.name",
                                      "task_id": "$subtasks_new.task_id",
                                      "job_started": True,
                                      "firstEvent": {"$first": "$subtasks_new.events"},
                                      "latestEvent": {"$last": "$subtasks_new.events"}}},
                        {'$project': {"_id": True, "name": True, "task_id": True, "job_started": True,
                                      "started": "$firstEvent.timestamp",
                                      "latest_update": "$latestEvent.timestamp",
                                      "status": "$latestEvent.state"
                                      }},
                        {'$sort': {"_id": 1, "started": 1}},
                        {"$group": {"_id": "$_id", "job_started": {"$max": "$job_started"},
                                    "subtasks": {"$push": {"task_id": "$task_id",
                                                               "name": "$name",
                                                               "status": "$status",
                                                               "started": "$started",
                                                               "latest_update": "$latest_update"}}}},
                        {"$sort": {"subtasks.started": 1}}
                        
                       ])

In [201]:
x = list(q)[0:3]
x

[{'_id': '0dc2a52c-ec0b-11ed-a46b-4c034f70a3f9',
  'job_started': 1683376773.800887,
  'subtasks': [{'task_id': 'd0893d9d-6f68-4f51-9384-09b88ccd88ce',
    'name': 'pyprotolinc.server.tasks.valuation_run_modular',
    'status': 'SUCCESS',
    'started': 1683376773.937037,
    'latest_update': 1683376773.956936},
   {'task_id': 'a7919b2c-9bcf-4361-b3a1-768dbab28ae0',
    'name': 'pyprotolinc.server.tasks.valuation_subportfolio',
    'status': 'SUCCESS',
    'started': 1683376773.9478524,
    'latest_update': 1683376774.948184},
   {'task_id': '27ae64d0-becd-4017-9fbc-7249df7966da',
    'name': 'pyprotolinc.server.tasks.valuation_subportfolio',
    'status': 'SUCCESS',
    'started': 1683376773.9480412,
    'latest_update': 1683376774.9482548},
   {'task_id': 'f3ecc959-0ec6-4b90-9d03-cc655cddc314',
    'name': 'pyprotolinc.server.tasks.valuation_subportfolio',
    'status': 'SUCCESS',
    'started': 1683376773.9570684,
    'latest_update': 1683376774.9583673},
   {'task_id': 'bdb1f5a4-fe

### Job Status Query

In [92]:
import pytz
from datetime import datetime

q = jobs_col.aggregate([{"$project": {"latestEvent": { "$last": "$events" }}},
                        {"$project": {"_id": 0, "job_id": "$_id", "state": "$latestEvent.state", "ts": "$latestEvent.timestamp"}}
                       ])

task_status = []
for job_info in list(q):
    task_status.append(job_info)
    ts = datetime.fromtimestamp(job_info["ts"], pytz.utc)
    job_info["ts"] = ts.strftime('%Y-%m-%d %H:%M:%S %Z%z')
    # ts.astimezone(pytz.timezone('Europe/Berlin')).strftime('%Y-%m-%d %H:%M:%S %Z%z')

task_status

[{'job_id': '16e6225d-69c9-4271-a988-9ab3d7d935f7',
  'state': 'SUCCESS',
  'ts': '2023-05-01 10:11:05 UTC+0000'}]

In [86]:
ts = datetime.fromtimestamp(r["events"][-1]['timestamp'], pytz.utc)
ts.astimezone(pytz.timezone('Europe/Berlin')).strftime('%Y-%m-%d %H:%M:%S %Z%z')

'2023-05-01 12:11:05 CEST+0200'

In [80]:
help(datetime.fromtimestamp)

Help on built-in function fromtimestamp:

fromtimestamp(...) method of builtins.type instance
    timestamp[, tz] -> tz's local time from POSIX timestamp.



In [99]:
from datetime import timezone
ts = datetime.now()
ts.replace(tzinfo=timezone.utc).timestamp()

1682949114.508494

In [100]:
help(datetime.now)

Help on built-in function now:

now(tz=None) method of builtins.type instance
    Returns new datetime object representing current time local to tz.
    
      tz
        Timezone object.
    
    If no tz is specified, uses local timezone.



In [103]:
ts = datetime.now(tz=timezone.utc).timestamp()
ts

1682941998.20702